## Modelagem de Tópicos com Latent Dirichlet Allocation (_LDA_)

A *modelagem de tópicos* é um conjunto de técnicas que podem ser usadas para descrever e resumir os documentos em um corpus de acordo com um conjunto de "tópicos" latentes. Para esta demonstração, usaremos o [*Latent Dirichlet Allocation*](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) ou o LDA, uma abordagem popular para a modelagem de tópicos.

Em muitos aplicativos convencionais de PNL, os documentos são representados por uma mistura de tokens individuais (palavras e frases) que eles contêm. Em outras palavras, um documento é representado como um vetor de contagens de tokens. Existem duas camadas neste modelo - documentos e tokens - e o tamanho ou dimensionalidade dos vetores do documento é o número de tokens no vocabulário do corpus. Essa abordagem tem várias desvantagens:
* Os vetores de documentos tendem a ser grandes (uma dimensão para cada token $ \ Rightarrow $ muitas dimensões)
* Eles também tendem a ser muito escassos. Qualquer documento fornecido contém apenas uma pequena fração de todos os tokens no vocabulário, portanto, a maioria dos valores no vetor de token do documento é 0.
* As dimensões são totalmente independentes uma da outra - não há senso de conexão entre os tokens relacionados, como faca e garfo.

O LDA injeta uma terceira camada nesse modelo conceitual. Os documentos são representados como uma mistura de um número predefinido de tópicos, e os tópicos são representados como uma mistura dos tokens individuais no vocabulário. O número de tópicos é um hiperparâmetro do modelo selecionado pelo praticante. A LDA faz uma suposição anterior de que as misturas (documento, tópico) e (tópico, token) seguem as distribuições de probabilidade de [*Dirichlet*](https://en.wikipedia.org/wiki/Dirichlet_distribution). Essa suposição incentiva os documentos a consistirem principalmente em um punhado de tópicos, e os tópicos consistem principalmente em um modesto conjunto de tokens.

![LDA](https://s3.amazonaws.com/skipgram-images/LDA.png)

O LDA é totalmente não supervisionado. Os tópicos são "descobertos" automaticamente a partir dos dados, tentando maximizar a probabilidade de observar os documentos em seu corpus, dadas as suposições de modelagem. Espera-se que eles capturem alguma estrutura latente e organização dentro dos documentos, e freqüentemente tenham uma interpretação humana significativa para pessoas familiarizadas com o material do assunto.

Voltaremos a gensim para ajudar na preparação e modelagem de dados. Em particular, o gensim oferece uma implementação paralela de alto desempenho do LDA com sua classe [**LdaMulticore**](https://radimrehurek.com/gensim/models/ldamulticore.html).

In [57]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.word2vec import LineSentence
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle
import itertools as it
from gensim.models import Phrases

import spacy

nlp = spacy.load('en')

O primeiro passo para criar um modelo de LDA é aprender o vocabulário completo do corpus a ser modelado. Usaremos a classe [**Dictionary**](https://radimrehurek.com/gensim/corpora/dictionary.html) do gensim para isso.

In [8]:
trigram_dictionary_filepath = 'trigram_dict_all.dict.txt'
trigram_reviews_filepath = 'trigram_reviews.txt'

In [9]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 1 == 1:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 472 ms, sys: 13.4 ms, total: 486 ms
Wall time: 508 ms


Como muitas técnicas de PNL, o LDA usa uma suposição simplificadora conhecida como modelo [*bag-of-words*](https://en.wikipedia.org/wiki/Bag-of-words_model). No modelo bag-of-words, um documento é representado pelas contagens de termos distintos que ocorrem dentro dele. Informações adicionais, como ordem de palavras, são descartadas.

Usando o dicionário gensim, aprendemos a gerar uma representação de saco de palavras para cada revisão. A função `trigram_bow_generator` implementa isso. Salvaremos as resenhas resultantes de saco de palavras como uma matriz.

No código a seguir, "bag-of-words" é abreviado como `bow`.

In [10]:
trigram_bow_filepath = 'trigram_bow_corpus_all.mm'

In [11]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [13]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 1 == 1:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 624 ms, sys: 14.3 ms, total: 639 ms
Wall time: 660 ms


Com o corpus bag-of-words, estamos finalmente prontos para aprender nosso modelo de tópico a partir dos comentários. Precisamos simplesmente passar a matriz bag of-words e o Dicionário de nossas etapas anteriores para o `LdaMulticore` como entradas, juntamente com o número de tópicos que o modelo deve aprender. Para esta demonstração, estamos pedindo 10 tópicos.

In [14]:
lda_model_filepath = 'lda_model_all.txt'

In [15]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 1 == 1:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=10,
                           id2word=trigram_dictionary,
                           workers=3)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 3.76 s, sys: 518 ms, total: 4.28 s
Wall time: 4.55 s


Nosso modelo de tópico está agora treinado e pronto para uso! Como cada tópico é representado como uma mistura de tokens, você pode inspecionar manualmente quais tokens foram agrupados em quais tópicos tentar entender os padrões que o modelo descobriu nos dados.

In [21]:
def explore_topic(topic_number, topn=10):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print ('{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=10):
        print ('{:20} {:.3f}'.format(term, round(frequency, 3)))

In [22]:
explore_topic(topic_number=0)

term                 frequency

order                0.013
great                0.011
do_not               0.011
fry                  0.009
table                0.009
place                0.009
like                 0.009
try                  0.007
come                 0.007
this_place           0.007


The first topic has strong associations with words like *taco*, *salsa*, *chip*, *burrito*, and *margarita*, as well as a handful of more general words. You might call this the **Mexican food** topic!

It's possible to go through and inspect each topic in the same way, and try to assign a human-interpretable label that captures the essence of each one. I've given it a shot for all 50 topics below.

O primeiro tópico tem associações com palavras como *order*, *great*, *do_not*, *fry* e *table*, bem como um punhado de palavras mais gerais.
É possível percorrer e inspecionar cada tópico da mesma maneira e tentar atribuir um rótulo interpretável por humanos que capte a essência de cada um. Eu dei uma chance para todos os 10 tópicos abaixo.

In [23]:
topic_names = {0: 'order',
               1: 'great',
               2: 'do_not',
               3: 'fry',
               4: 'table',
               5: 'place',
               6: 'like',
               7: 'try',
               8: 'come',
               9: 'this_place',
               }

In [26]:
topic_names_filepath = 'topic_names.pkl'

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

Rever manualmente os principais termos para cada tópico é um exercício útil, mas para obter uma compreensão mais profunda dos tópicos e como eles se relacionam entre si, precisamos visualizar os dados - de preferência em um formato interativo. Felizmente, temos a fantástica biblioteca [**pyLDAvis**](https://pyldavis.readthedocs.io/en/latest/readme.html) para ajudar com isso!

O pyLDAvis inclui uma função de uma linha para pegar modelos de tópicos criados com o gensim e preparar seus dados para visualização.

In [27]:
LDAvis_data_filepath = 'ldavis_prepared.txt'

In [29]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

CPU times: user 3.68 s, sys: 57.4 ms, total: 3.74 s
Wall time: 4.22 s


`pyLDAvis.display(...)` exibe a visualização do modelo de tópico in-line no notebook.

In [30]:
pyLDAvis.display(LDAvis_prepared)

### Espere, o que estou olhando de novo?
Há muitas partes na visualização. Aqui está um breve resumo:

* À esquerda, há um gráfico da "distância" entre todos os tópicos (rotulados como o Mapa de Distância Interp.)
    * A plotagem é renderizada em duas dimensões, de acordo com um algoritmo de escalonamento multidimensional [*multidimensional scaling (MDS)*](https://en.wikipedia.org/wiki/Multidimensional_scaling). Tópicos que geralmente são semelhantes devem aparecer juntos no enredo, enquanto tópicos diferentes devem aparecer distantes.
    * O tamanho relativo do círculo de um tópico no gráfico corresponde à frequência relativa do tópico no corpus.
    * Um tópico individual pode ser selecionado para uma análise mais detalhada, clicando em seu círculo ou inserindo seu número na caixa "tópico selecionado" no canto superior esquerdo.
* À direita, há um gráfico de barras mostrando os principais termos.
    * Quando nenhum tópico é selecionado na plotagem à esquerda, o gráfico de barras mostra os 30 principais termos "salientes" no corpus. A saliência de um termo é uma medida de quão freqüente o termo está no corpus e de quão "distintivo" é distinguir entre diferentes tópicos.
    * Quando um determinado tópico é selecionado, o gráfico de barras é alterado para mostrar os 30 principais termos "relevantes" para o tópico selecionado. A métrica de relevância é controlada pelo parâmetro λ
 , que pode ser ajustado com um controle deslizante acima do gráfico de barras.
        * Configurando o parâmetro $\lambda$
   próximo a 1.0 (o padrão) classificará os termos unicamente de acordo com sua probabilidade dentro do tópico.
        * Configurando $\lambda$
  próximo de 0.0 classificará os termos unicamente de acordo com sua "distinção" ou "exclusividade" no tópico - isto é, termos que ocorrem apenas neste tópico e não ocorrem em outros tópicos.
        * Configurando $\lambda$
  valores entre 0.0 e 1.0 resultarão em uma classificação intermediária, ponderando a probabilidade de termos e a exclusividade de acordo.
* Rolar o mouse sobre um termo no gráfico de barras à direita fará com que os círculos de tópicos sejam redimensionados na plotagem à esquerda, para mostrar a força da relação entre os tópicos e o termo selecionado.

Uma explicação mais detalhada da visualização do pyLDAvis pode ser encontrada [aqui](https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf). Infelizmente, embora os dados usados pelo gensim e pelo pyLDAvis sejam os mesmos, eles não usam os mesmos números de identificação para os tópicos. Se você precisa combinar tópicos no objeto `LdaMulticore` do gensim e na visualização do pyLDAvis, você tem que vasculhar os termos manualmente.

### Analisando nosso modelo de LDA
A visualização interativa que o pyLDAvis produz é útil para ambos:
1. Melhor compreensão e interpretação de tópicos individuais e
1. Compreender melhor as relações entre os tópicos.

Para (1), você pode selecionar manualmente cada tópico para visualizar seus termos mais frequentes e / ou "relevantes", usando valores diferentes do parâmetro $\lambda$. Isso pode ajudar quando você está tentando atribuir um nome interpretável ou um "significado" a cada tópico.

Para (2), explorar o _Intertopic Distance Plot_ pode ajudá-lo a aprender sobre como os tópicos se relacionam entre si, incluindo a estrutura potencial de alto nível entre grupos de tópicos.

### Descrevendo texto com LDA
Além da exploração de dados, um dos principais usos para um modelo de LDA é fornecer uma descrição compacta e quantitativa do texto da linguagem natural. Uma vez que um modelo de LDA tenha sido treinado, ele pode ser usado para representar texto livre como uma mistura dos tópicos que o modelo aprendeu com o corpus original. Essa mistura pode ser interpretada como uma distribuição de probabilidade entre os tópicos, portanto, a representação de um parágrafo do texto em LDA pode parecer 50% do tópico A, 20% do tópico B, 20% do tópico C e 10% do tópico D.

Para usar um modelo de LDA para gerar uma representação vetorial de novo texto, você precisará aplicar as etapas de pré-processamento de texto usadas no corpus de treinamento do modelo ao novo texto também. Para o nosso modelo, as etapas de pré-processamento utilizadas incluem:
1. Usando spaCy para remover pontuação e lematizar o texto
1. Aplicando nosso modelo de frase de primeira ordem para unir pares de palavras
1. Aplicando nosso modelo de frase de segunda ordem para juntar frases mais longas
1. Removendo stopwords
1. Criando uma representação bag-of-words

Depois de aplicar essas etapas de pré-processamento ao novo texto, ele estará pronto para passar diretamente ao modelo para criar uma representação de LDA. A função `lda_description (...)` executará todas essas etapas para nós, incluindo a impressão da descrição tópica resultante do texto de entrada.

In [51]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review('review.txt'),
                          review_number, review_number+1))[0]

def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')

In [61]:
bigram_model = Phrases.load('bigram_model.txt')
trigram_model = Phrases.load('trigram_model.txt')

In [65]:

def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review
                      if not term in spacy.lang.en.STOP_WORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda tupla: -tupla[1])
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print ('{:25} {}'.format(topic_names[topic_number],
                                round(freq, 3)))

In [66]:
sample_review = get_sample_review(50)
print (sample_review)

This past weekend, my bf and I did a quick trip to Champaign, because we just needed to get out of Chicago for a couple of days.  When we parked our car and walked down Green St., there were a whole bunch of NEW restaurants compared to when I went to college years ago.  One familiar restaurant was Zorba's.  Got two med. gyros, fries, and drinks to go.  When we sat outside on the quad, we upwrapped a tightly rolled gyro.  So eating it outside on a windy day was not a problem at all!  Really good and the pita seemed fresh and light, not super greasy like the joints here in Chicago.  Don't think I'll go back to Champaign for a long time, but it's a great lunch/dinner option.



In [67]:
lda_description(sample_review)

order                     0.6679999828338623
like                      0.16300000250339508
place                     0.15600000321865082


/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [71]:
sample_review = get_sample_review(120)
print (sample_review)

Facility is similar to a Dave and Busters but with a larger arcade area and a bar instead of restaurant.

Games: Okay selection. Typical games like carnival style, ticket winning games, arcade racing, two rhythm and very few Japanese games. My personal favorite is Taiko Drum Master. Some o the games are rather old and as maintain as other places.

People: For families and children. The kids could be worst and parents have them in line as best they can. Something about the place and the energy causes the parents to be very impatient and irrational.  Probably some kids are pestering their parents and they are at their wits end. The parents are only interested with their kids and they will interrupt your game if their kids want to play. Teenagers are just as bad. "Every man from himself" is what this place is like. That happen twice in one visit.

(Specific example, we waited in like for about 1/2-1hr for a game. Then a parent interrupted us when we were playing the game in the beginning 

In [72]:
lda_description(sample_review)

order                     0.5379999876022339
like                      0.3499999940395355
this_place                0.10499999672174454


/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
